In [2]:
import torch
import torch.nn.utils.prune as prune 
import numpy as np
import plotly.graph_objects as go 
import pandas as pd 
import torchvision
from torch.autograd import Variable


In [3]:
check_gpu = torch.cuda.is_available()
check_gpu

True

In [4]:
check_devices = torch.cuda.get_device_name(0)
check_devices

'NVIDIA GeForce RTX 2080 Ti'

In [5]:
crypto_name = 'apeusd'
#example_ohlc_df = pd.DataFrame({"Time": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"Open": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"High": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"Low": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"Close": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]})

ohlc_df = pd.read_csv('historical_price/' + crypto_name + '.csv')
ohlc_df.index = pd.to_datetime(ohlc_df['time'], unit = 'ms')

#ohlc_df.index = ohlc_df.index.tz_localize('UTC').tz_convert('US/Eastern')

In [6]:
price_dataset = ohlc_df.close[-10000:]
price_dataset.head()

time
2022-06-12 04:56:00    4.2028
2022-06-12 04:57:00    4.2043
2022-06-12 04:58:00    4.1887
2022-06-12 04:59:00    4.1873
2022-06-12 05:00:00    4.1954
Name: close, dtype: float64

In [ ]:
train_loader = torch.utils.data.DataLoader(
    root = 'historical_price/' + crypto_name + '.csv',
    train=True, 
    transform=torchvision.transforms.Compose,
    batch_size=1, 
    shuffle=False)

test_loader = torch.utils.data.DataLoader( 
    root = 'historical_price/' + crypto_name + '.csv',
    train=False,
    transform=torchvision.transforms.Compose,
    batch_size=1,
    shuffle=False)


In [6]:
class LSTM_Model(torch.nn.Module):
    def __init__(self, batch_size, output_size, hidden_size, embedding_length):
        super(LSTM_Model, self).__init__()
        self.batch_size = batch_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.lstm = torch.nn.LSTM(embedding_length, hidden_size) # Our main hero for this tutorial
        self.label = torch.nn.Linear(hidden_size, output_size)
        
    def forward(self, input_sentence, batch_size=None):
        input = self.word_embeddings(input_sentence) 
        input = input.permute(1, 0, 2) 
        if batch_size is None:
            h_0 = Variable(torch.zeros(1, self.batch_size, self.hidden_size).cuda()) 
            c_0 = Variable(torch.zeros(1, self.batch_size, self.hidden_size).cuda()) 
        else:
            h_0 = Variable(torch.zeros(1, batch_size, self.hidden_size).cuda())
            c_0 = Variable(torch.zeros(1, batch_size, self.hidden_size).cuda())
        output, (final_hidden_state, final_cell_state) = self.lstm(input, (h_0, c_0))
        final_output = self.label(final_hidden_state[-1]) 
        
        return final_output
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
crypto_price_model = LSTM_Model()
if torch.cuda.is_available():
    crypto_price_model.cuda()
crypto_price_model.to(device)

loss_function = torch.nn.CrossEntropyLoss() # Criterion, I believe CrossEntropyLoss has SoftMax built-in so I used it here, this is why my last layer doesn't have a softmax
optimizer  = torch.optim.Adam(crypto_price_model.parameters(), lr=0.001) # used adaptive moment estimation to optimize the model

cuda


In [ ]:
def clip_gradient(model, clip_value):
    params = list(filter(lambda p: p.grad is not None, model.parameters()))
    for p in params:
        p.grad.data.clamp_(-clip_value, clip_value)
    
def train_model(model, train_iter, epoch):
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.cuda()
    optim = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))
    steps = 0
    model.train()
    for idx, batch in enumerate(train_iter):
        text = batch.text[0]
        target = batch.label
        target = torch.autograd.Variable(target).long()
        if torch.cuda.is_available():
            text = text.cuda()
            target = target.cuda()
        if (text.size()[0] is not 32): 
            continue
        optim.zero_grad()
        prediction = model(text)
        loss = loss_fn(prediction, target)
        wandb.log({"Training Loss": loss.item()})
        num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).float().sum()
        acc = 100.0 * num_corrects/len(batch)
        wandb.log({"Training Accuracy": acc.item()})
        loss.backward()
        clip_gradient(model, 1e-1)
        optim.step()
        steps += 1
        
        if steps % 100 == 0:
            print (f'Epoch: {epoch+1}, Idx: {idx+1}, Training Loss: {loss.item():.4f}, Training Accuracy: {acc.item(): .2f}%')
        
        total_epoch_loss += loss.item()
        total_epoch_acc += acc.item()
        
    return total_epoch_loss/len(train_iter), total_epoch_acc/len(train_iter)

def eval_model(model, val_iter):
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for idx, batch in enumerate(val_iter):
            text = batch.text[0]
            if (text.size()[0] is not 32):
                continue
            target = batch.label
            target = torch.autograd.Variable(target).long()
            if torch.cuda.is_available():
                text = text.cuda()
                target = target.cuda()
            prediction = model(text)
            loss = loss_fn(prediction, target)
            wandb.log({"Evaluation Loss": loss.item()})
            num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).sum()
            acc = 100.0 * num_corrects/len(batch)
            wandb.log({"Evaluation Accuracy": acc.item()})
            total_epoch_loss += loss.item()
            total_epoch_acc += acc.item()

    return total_epoch_loss/len(val_iter), total_epoch_acc/len(val_iter)

In [ ]:
torch.save(crypto_price_model.state_dict(), f'{crypto_name}_model_weights.pth')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
crypto_price_model.load_state_dict(torch.load(f'{crypto_name}_model_weights.pth', map_location=torch.device(device)))